In [1]:

from transformers import LlavaOnevisionForConditionalGeneration, LlavaOnevisionProcessor
from utils import MultiModalDataset, MultiModalCollator
import torch
from torch.utils.data import Dataset
from PIL import Image
import av
import numpy as np
from dataclasses import dataclass
from typing import List, Dict, Any
import os


from trainer import print_LiME_summary, LiMEArguments, LiMETrainer

from transformers import TrainingArguments
from evaluation import run_full_evaluation

from datasets import load_from_disk
from transformers import TrainingArguments, Trainer

    
from torch import nn



from LiMELoRAFA import apply_peft

cache_dir = "/ocean/projects/cis250258p/mkowsher/hf_cache"
model_name = "/ocean/projects/cis250258p/mkowsher/hf_cache/llava-onevision-qwen2-7b-ov-hf"
save_csv="evaluation_results_lorafa.csv"
data_root="/ocean/projects/cis250258p/mkowsher/dataset/MVBench"
dataset_name="hf_mvbench_updated"

# Load processor
processor = LlavaOnevisionProcessor.from_pretrained(model_name, cache_dir=cache_dir)

# ⭐ CRITICAL: Set fixed resolution BEFORE using processor
processor.image_processor.image_grid_pinpoints = [[384, 384]]
processor.image_processor.size = {"height": 384, "width": 384}
processor.tokenizer.padding_side = "left" 

if processor.tokenizer.pad_token is None:
    processor.tokenizer.pad_token = processor.tokenizer.eos_token

# Load model
model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    cache_dir=cache_dir,
)


# ⭐ CRITICAL: Update model config to match
model.config.image_grid_pinpoints = [[384, 384]]

# ⭐ VERIFY the settings
print("="*50)
print("VERIFICATION:")
print("="*50)
print(f"Processor image_grid_pinpoints: {processor.image_processor.image_grid_pinpoints}")
print(f"Model config image_grid_pinpoints: {model.config.image_grid_pinpoints}")
print(f"Vision config image_size: {model.config.vision_config.image_size}")
print("="*50)

model.gradient_checkpointing_enable()

# Load dataset
dataset = load_from_disk(dataset_name)

train_dataset = MultiModalDataset(
    dataset=dataset['train'],
    processor=processor,
    data_root=data_root,
    num_video_frames=8,
    max_length=2048,
)
collator = MultiModalCollator(
    processor=processor,
    max_length=2048,
)

# ============ TEST FIRST ============
print("\n" + "="*50)
print("TESTING SINGLE SAMPLES:")
print("="*50)

# Test one of each modality
for i in range(min(50, len(train_dataset))):
    sample = train_dataset[i]
    src_type = sample.get('source_type', 'unknown')
    print(f"\nSample {i} ({src_type}):")
    print(f"  input_ids: {sample['input_ids'].shape}")
    if 'pixel_values' in sample:
        print(f"  pixel_values: {sample['pixel_values'].shape}")
    if 'image_sizes' in sample:
        print(f"  image_sizes: {sample['image_sizes']}")
    if 'pixel_values_videos' in sample:
        print(f"  pixel_values_videos: {sample['pixel_values_videos'].shape}")
    
    # Stop after finding one of each
    if i > 10:
        break

# Test collator with small batch
print("\n" + "="*50)
print("TESTING COLLATOR:")
print("="*50)

test_samples = [train_dataset[i] for i in range(4)]
test_batch = collator(test_samples)
print("Batch keys:", list(test_batch.keys()))
for k, v in test_batch.items():
    if isinstance(v, torch.Tensor):
        print(f"  {k}: {v.shape}")



import torch

# Collect exactly 20 samples from the PROCESSED dataset
samples = []
for i in range(20):
    samples.append(train_dataset[i])  # ✅ Use train_dataset, not dataset['train']

# Track modalities before collating
modalities = [s['source_type'] for s in samples]

# Run through collator
batch = collator(samples)

# Inspect tensor shapes
print("="*70)
print("BATCH TENSOR SHAPES:")
print("="*70)
for k, v in batch.items():
    if isinstance(v, torch.Tensor):
        print(f"{k}: {v.shape}, dtype={v.dtype}")
    elif isinstance(v, list):
        print(f"{k}: list of {len(v)} items")

# Check modalities
print("\n" + "="*70)
print("SAMPLE MODALITIES:")
print("="*70)
for i, mod in enumerate(modalities):
    print(f"Sample {i}: {mod}")

# Count by modality
from collections import Counter
mod_counts = Counter(modalities)
print(f"\nModality distribution: {dict(mod_counts)}")

# Check visual data
print("\n" + "="*70)
print("VISUAL DATA INFO:")
print("="*70)
if 'pixel_values' in batch:
    print(f"pixel_values (images): {batch['pixel_values'].shape}")
    print(f"  - Number of image samples in batch: {modalities.count('image')}")
else:
    print("No images in this batch")

if 'pixel_values_videos' in batch:
    print(f"pixel_values_videos: {batch['pixel_values_videos'].shape}")
    print(f"  - Number of video samples in batch: {modalities.count('video')}")
else:
    print("No videos in this batch")

if 'image_sizes' in batch:
    print(f"image_sizes: {batch['image_sizes'].shape}")

# Decode and inspect
print("\n" + "="*70)
print("DECODED TEXT (last 150 tokens of each sample):")
print("="*70)

for i in range(min(10, len(samples))):  # First 10 for readability
    modality = modalities[i]
    
    # Get last 150 tokens
    last_tokens = batch["input_ids"][i][-150:]
    decoded = processor.tokenizer.decode(last_tokens, skip_special_tokens=False)
    
    # Get labels
    labels = batch["labels"][i]
    label_tokens = labels[labels != -100]
    label_text = processor.tokenizer.decode(label_tokens, skip_special_tokens=False) if len(label_tokens) > 0 else "[NO LABELS]"
    
    print(f"\n{'='*70}")
    print(f"SAMPLE {i} | Modality: {modality}")
    print(f"{'='*70}")
    print(f"Last 150 tokens decoded:\n{decoded}")
    print(f"\n📝 LABELS (what model learns to predict):\n{label_text}")
    print(f"Label token count: {len(label_tokens)}")

# Verify token-feature alignment for images
print("\n" + "="*70)
print("TOKEN-FEATURE ALIGNMENT CHECK:")
print("="*70)
image_token_id = processor.tokenizer.convert_tokens_to_ids("<image>")
video_token_id = processor.tokenizer.convert_tokens_to_ids("<video>")

total_image_tokens = (batch['input_ids'] == image_token_id).sum().item()
total_video_tokens = (batch['input_ids'] == video_token_id).sum().item()

print(f"Total <image> tokens in batch: {total_image_tokens}")
print(f"Total <video> tokens in batch: {total_video_tokens}")

if 'pixel_values' in batch:
    # For fixed 384x384, each image = 1 patch = 729 features (27x27) after pooling
    # But depends on model config
    print(f"pixel_values shape: {batch['pixel_values'].shape}")
    
if 'pixel_values_videos' in batch:
    print(f"pixel_values_videos shape: {batch['pixel_values_videos'].shape}")
    
    


targets = ["q_proj", "k_proj", "v_proj", "o_proj", "out_proj"]


def lm_only_targets(path, module):
    # Only wrap modules under model.language_model
    if not path.startswith("model.language_model."):
        return False

    # Expect paths like: model.language_model.layers.<idx>.*
    if ".layers." not in path:
        return False

    try:
        layer_id = int(path.split(".layers.")[1].split(".")[0])
    except (IndexError, ValueError):
        return False


    return any(path.endswith(name) for name in targets)

model = apply_peft(
    model,
    targets=targets,
    num_experts=4,
    rank=2,
    use_shared_LiME=True,
    n_gram=2,
    top_k=1,
    rep_mode="token",
    jitter_noise=0.1,
    tokenizer=processor.tokenizer,
    temperature=0.5,
    gamma_routing = 0.7, 
    auto_topk=True, 
    auto_topk_threshold=0.5, 
    peft_dtype=torch.float32,   # A, B in float32
    moe_dtype=torch.float32,    # moe3s, gamma in float32

)

print_LiME_summary(model)

# 1) Count params that require grad
trainable = [(n, p) for n, p in model.named_parameters() if p.requires_grad]
total = sum(p.numel() for _, p in model.named_parameters())
trainable_num = sum(p.numel() for _, p in trainable)
print(f"trainable params: {trainable_num:,} / {total:,}")




#model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
training_args = LiMEArguments(
    output_dir="./llava-lora-finetuned",
    per_device_train_batch_size=5,
    gradient_accumulation_steps=4,  
    save_total_limit=2,
    save_steps=500000,
    num_train_epochs=2,
    bf16=True,  
    logging_dir="./logs",
    logging_steps=100,
    remove_unused_columns=False, 

    eval_steps=100,
    save_strategy="steps",
    optim="adamw_bnb_8bit",
    learning_rate=2e-4,
    warmup_ratio=0.03,
    weight_decay=0.01,
    report_to="none",
    disable_tqdm=False,          # makes it print log lines instead of tqdm bar behavior
    #log_level="info",
    #logging_first_step=True,
    moe_lr=1e-3,          # For propulsions, gamma (float32)
    peft_lr=4e-4,         # For LoRA A/B (float32)
    importance_coef=0.1,
    kl_coef=0.01,
  
)

# Also make sure model doesn't have it enabled
model.gradient_checkpointing_disable()  # ⭐ Call this explicitly



# Example instantiation:
trainer = LiMETrainer(
    model=model,
    args=training_args,                  # your HF TrainingArguments
    train_dataset=train_dataset,

    tokenizer=processor.tokenizer,
    data_collator=collator,  # ✅ Custom collator dynamically pads batch sequences


)
trainer.train() 



    





/jet/home/mkowsher/.conda/envs/myenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.73s/it]
/jet/home/mkowsher/.conda/envs/myenv/lib/python3.13/site-packages/accelerate/utils/modeling.py:1598: UserWarning: The following device_map keys do not match any submodules in the model: ['model.image_newline']
  warnings.warn(


VERIFICATION:
Processor image_grid_pinpoints: [[384, 384]]
Model config image_grid_pinpoints: [[384, 384]]
Vision config image_size: 384

TESTING SINGLE SAMPLES:

Sample 0 (image):
  input_ids: torch.Size([1509])
  pixel_values: torch.Size([2, 3, 384, 384])
  image_sizes: tensor([512, 512])

Sample 1 (text):
  input_ids: torch.Size([59])

Sample 2 (image):
  input_ids: torch.Size([1508])
  pixel_values: torch.Size([2, 3, 384, 384])
  image_sizes: tensor([512, 512])

Sample 3 (video):
  input_ids: torch.Size([1614])
  pixel_values_videos: torch.Size([8, 3, 384, 384])

Sample 4 (image):
  input_ids: torch.Size([1525])
  pixel_values: torch.Size([2, 3, 384, 384])
  image_sizes: tensor([512, 512])

Sample 5 (text):
  input_ids: torch.Size([50])

Sample 6 (image):
  input_ids: torch.Size([1522])
  pixel_values: torch.Size([2, 3, 384, 384])
  image_sizes: tensor([32, 32])

Sample 7 (text):
  input_ids: torch.Size([83])

Sample 8 (text):
  input_ids: torch.Size([48])

Sample 9 (text):
  input

/ocean/projects/cis250258p/mkowsher/dataset/MVBench/trainer.py:337: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `LiMETrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


BATCH TENSOR SHAPES:
input_ids: torch.Size([20, 1614]), dtype=torch.int64
attention_mask: torch.Size([20, 1614]), dtype=torch.int64
labels: torch.Size([20, 1614]), dtype=torch.int64
pixel_values: torch.Size([10, 3, 384, 384]), dtype=torch.float32
image_sizes: torch.Size([5, 2]), dtype=torch.int64
pixel_values_videos: torch.Size([3, 8, 3, 384, 384]), dtype=torch.float32

SAMPLE MODALITIES:
Sample 0: image
Sample 1: text
Sample 2: image
Sample 3: video
Sample 4: image
Sample 5: text
Sample 6: image
Sample 7: text
Sample 8: text
Sample 9: text
Sample 10: text
Sample 11: video
Sample 12: text
Sample 13: text
Sample 14: text
Sample 15: text
Sample 16: video
Sample 17: text
Sample 18: image
Sample 19: text

Modality distribution: {'image': 5, 'text': 12, 'video': 3}

VISUAL DATA INFO:
pixel_values (images): torch.Size([10, 3, 384, 384])
  - Number of image samples in batch: 5
pixel_values_videos: torch.Size([3, 8, 3, 384, 384])
  - Number of video samples in batch: 3
image_sizes: torch.Size(

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,6.262500
200,0.485800
300,0.211300
400,0.207000
500,0.193400
600,0.179200
700,0.167500
800,0.182900
900,0.170100
1000,0.175200


TrainOutput(global_step=15862, training_loss=0.15963918629758012, metrics={'train_runtime': 90320.8397, 'train_samples_per_second': 3.512, 'train_steps_per_second': 0.176, 'total_flos': 2.035490262486711e+19, 'train_loss': 0.15963918629758012, 'task_loss': 0.009864027611911297, 'balance_loss': 0.0005144942551851273, 'epoch': 2.0})

In [2]:
from evaluation import run_full_evaluation
results = run_full_evaluation(
    model=model,
    processor=processor,
    dataset=dataset,
    data_root=data_root,
    num_samples_per_split=1000,
    batch_size=6,
    max_new_tokens=50, 
    save_csv=save_csv,
)



EVALUATION: 47 splits, 1000 samples each, batch_size=6
Numeric tolerance: 0.5

Evaluating: image_test_chartqa (1000 samples, batch_size=6)
Task type: classification, Numeric tolerance: 0.5


image_test_chartqa: 100%|██████████| 167/167 [03:20<00:00,  1.20s/it]



📊 image_test_chartqa: 750/1000 = 75.00%
   Match types: {'no_match': 250, 'exact': 688, 'numeric': 58, 'contains': 4}
   ❌ True: '15.84' | Pred: '15.09' | Raw: '15.09' 
   ✅ True: '146' | Pred: '146' | Raw: '146' [exact]
   ❌ True: '10.37' | Pred: '11.32' | Raw: '11.32' 
   ❌ True: '14722.84' | Pred: '14738.81' | Raw: '14738.81' 
   ✅ True: '75.82' | Pred: '75.82' | Raw: '75.82' [exact]

Evaluating: image_test_okvqa (841 samples, batch_size=6)
Task type: multiple_choice, Numeric tolerance: 0.5


image_test_okvqa: 100%|██████████| 141/141 [02:20<00:00,  1.01it/s]



📊 image_test_okvqa: 489/841 = 58.15%
   Match types: {'exact': 464, 'no_match': 352, 'contains': 24, 'numeric': 1}
   ✅ True: 'river' | Pred: 'river' | Raw: 'river' [exact]
   ❌ True: 'carlo collodi' | Pred: 'gustave' | Raw: 'gustave flaubert' 
   ❌ True: '1936' | Pred: '1938' | Raw: '1938' 
   ✅ True: 'airplane' | Pred: 'airplane' | Raw: 'airplane' [exact]
   ✅ True: 'recreational' | Pred: 'recreational' | Raw: 'recreational' [exact]

Evaluating: image_test_scienceqa (518 samples, batch_size=6)
Task type: classification, Numeric tolerance: 0.5


...


📊 video_test_egocentric_navigation: 187/300 = 62.33%
   Match types: {'no_match': 113, 'exact': 187}
   ❌ True: 'e' | Pred: 'd' | Raw: 'D' 
   ✅ True: 'e' | Pred: 'e' | Raw: 'E' [exact]
   ✅ True: 'a' | Pred: 'a' | Raw: 'A' [exact]
   ✅ True: 'b' | Pred: 'b' | Raw: 'B' [exact]
   ✅ True: 'b' | Pred: 'b' | Raw: 'B' [exact]

Evaluating: video_test_episodic_reasoning (300 samples, batch_size=6)
Task type: classification, Numeric tolerance: 0.5


video_test_episodic_reasoning: 100%|██████████| 50/50 [01:41<00:00,  2.04s/it]



📊 video_test_episodic_reasoning: 93/300 = 31.00%
   Match types: {'no_match': 207, 'exact': 93}
   ❌ True: 'c' | Pred: 'e' | Raw: 'E' 
   ✅ True: 'a' | Pred: 'a' | Raw: 'A' [exact]
   ✅ True: 'a' | Pred: 'a' | Raw: 'A' [exact]
   ❌ True: 'f' | Pred: 'b' | Raw: 'B' 
   ❌ True: 'd' | Pred: 'e' | Raw: 'E' 

Evaluating: video_test_counterfactual_inference (300 samples, batch_size=6)
Task type: classification, Numeric tolerance: 0.5


video_test_counterfactual_inference: 100%|██████████| 50/50 [01:03<00:00,  1.27s/it]


📊 video_test_counterfactual_inference: 261/300 = 87.00%
   Match types: {'exact': 261, 'no_match': 39}
   ✅ True: 'f' | Pred: 'f' | Raw: 'F' [exact]
   ✅ True: 'd' | Pred: 'd' | Raw: 'D' [exact]
   ✅ True: 'd' | Pred: 'd' | Raw: 'D' [exact]
   ❌ True: 'd' | Pred: 'e' | Raw: 'E' 
   ✅ True: 'f' | Pred: 'f' | Raw: 'F' [exact]

SUMMARY

📁 GLUE_TEST:
   Classification: 3684/4280 = 86.07%
     - glue_test_sst2: 94.84%
     - glue_test_qnli: 94.60%
     - glue_test_qqp: 86.10%
     - glue_test_cola: 69.10%
     - glue_test_mrpc: 87.99%
   Regression: Pearson=0.9196, Spearman=0.9213
     - glue_test_stsb: Pearson=0.9196, Spearman=0.9213

📁 IMAGE_TEST:
   Classification: 7019/8476 = 82.81%
     - image_test_chartqa: 75.00%
     - image_test_okvqa: 58.15%
     - image_test_scienceqa: 98.26%
     - image_test_seed_bench: 76.40%
     - image_test_text_recognition: 88.20%
     - image_test_textvqa: 72.10%
     - image_test_vizwiz_vqa: 68.82%
     - image_test_vqa_rad: 83.00%
     - image_test_cal